In [114]:
from hashlib import sha256
import time

In [125]:
class Block:
    def __init__(self, index, data, timestamp, previous_hash):
        self.index = index
        self.data = data
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.hash = 0
        self.nonce = 0
        
    def show(self):
        # A function to print the block.
       
        print("| index: "+str(self.index)+" >> {data:"+str(self.data)+
              ", timestamp:"+str(self.timestamp)+", previous_hash:"+
              str(self.previous_hash)+", block_hash:"+str(self.hash)+", nonce:"+
              str(self.nonce)+"} |")

    def compute_hash(self):
        # A function to compute a hash of the block the function is called by.
        
        block_string = (str(self.index) + str(self.data) + 
                        str(self.timestamp) + str(self.previous_hash) + 
                        str(self.nonce))
        return sha256(block_string.encode()).hexdigest()
    

In [126]:
test_block = Block(0, 0, 0, 0)
test_block.show()

| index: 0 >> {data:0, timestamp:0, previous_hash:0, block_hash:0, nonce:0} |


In [130]:
class Blockchain:
    
    difficulty = 2

    @property
    def last_block(self):
        return self.chain[-1] 

    def __init__(self):
        self.chain = []
        self.create_genesis_block()
        
    def show_chain(self):
        # A function to print the chain
        
        for i in self.chain:
            i.show()
            
    def verify_chain(self):
        # A function to verify the whole chain, checking the validity of each block

        if(len(self.chain) <= 2):
            return True
        else:
            for i in range(len(self.chain)-1, 1, -1):
                block = self.chain[i]
                pre_block = self.chain[i-1]
                if not self.verify_chainlink(block, pre_block):
                    return False
            return True
        
    def verify_chainlink(self, block, previous_block):
        # A function to verify a single chain link
        
        return ((block.hash).startswith('0' * Blockchain.difficulty) and
                (block.hash) == block.compute_hash() and 
                block.previous_hash == previous_block.compute_hash())

    def create_genesis_block(self):
        # A function to generate genesis block and appends it to
        # the chain. The block has index 0, previous_hash as 0, and
        # a valid hash.
        
        genesis_block = Block(0, [], time.time(), "0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)

    def proof_of_work(self, block, show_process=False):
        # Function that tries different values of nonce to get a hash
        # that satisfies our difficulty criteria.
        block.nonce = 0

        computed_hash = block.compute_hash()
        if not show_process:
            while not computed_hash.startswith('0' * Blockchain.difficulty):
                block.nonce += 1
                computed_hash = block.compute_hash()
        else:
            tries_num = 0
            while not computed_hash.startswith('0' * Blockchain.difficulty):
                block.nonce += 1
                computed_hash = block.compute_hash()
                print("Try number: "+str(tries_num)+" -> "+computed_hash)
                tries_num += 1

        return computed_hash

    def add_block(self, block, proof):
        # A function that adds the block to the chain after verification.
        
        previous_hash = self.last_block.hash

        if previous_hash != block.previous_hash:
            return False

        if not self.is_valid_proof(block, proof):
            return False

        block.hash = proof
        self.chain.append(block)
        return True

    def is_valid_proof(self, block, block_hash):
        # Check if block_hash is valid hash of block and satisfies
        # the difficulty criteria.
        
        return (block_hash.startswith('0' * Blockchain.difficulty) and
                block_hash == block.compute_hash())

    def mine(self, new_data, show_process=False):
        # This function mines a new block containing the
        # data given, if show_process is set as true the
        # nonce finding process is shown.
        
        last_block = self.last_block

        new_block = Block(index=last_block.index + 1,
                          data=new_data,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)

        proof = self.proof_of_work(new_block, show_process)
        self.add_block(new_block, proof)
        return new_block.index


In [129]:
test_chain = Blockchain()
test_chain.mine("new data")
test_chain.mine("new data2")
test_chain.show_chain()
test_chain.verify_chain()

| index: 0 >> {data:[], timestamp:1556150135.6601815, previous_hash:0, block_hash:22c85d3ade12cb87241209039f27f5b88715a78fe5b96becbb89b67ae207a6c1, nonce:0} |
| index: 1 >> {data:new data, timestamp:1556150135.6602893, previous_hash:22c85d3ade12cb87241209039f27f5b88715a78fe5b96becbb89b67ae207a6c1, block_hash:00dd578f199ec7323e22ffd7631496deb89c042034d039e5895a1a52d8abc03f, nonce:786} |
| index: 2 >> {data:new data2, timestamp:1556150135.6651208, previous_hash:00dd578f199ec7323e22ffd7631496deb89c042034d039e5895a1a52d8abc03f, block_hash:0013f49c128f99d9f2acbeb4757de2ac00472d7c0161d047d9bbfba9ac8043aa, nonce:503} |


True